You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of the growing importance of Artificial Intelligence in various industries
- Mention of the purpose of the article to highlight the latest trends and key players in the field

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advancements in machine learning algorithms
- Rise of explainable AI
- Increased focus on ethical AI practices
- Integration of AI in healthcare, financ

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become increasingly vital across various industries, revolutionizing processes and driving innovation. In this article, we aim to shed light on the latest trends and key players shaping the AI landscape. From advancements in machine learning algorithms to ethical considerations in AI development, we will explore the dynamic world of Artificial Intelligence.

## Latest Trends in Artificial Intelligence

The field of AI is constantly evolving, with new trends emerging to enhance its capabilities. One significant trend is the advancements in machine learning algorithms, enabling AI systems to learn and adapt to complex data patterns more efficiently. Additionally, there is a growing emphasis on explainable AI, where AI systems provide transparent explanations for their decisions, increasing trust and understanding among us

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become increasingly vital across various industries, revolutionizing processes and driving innovation. In this article, we aim to shed light on the latest trends and key players shaping the AI landscape. From advancements in machine learning algorithms to ethical considerations in AI development, we will explore the dynamic world of Artificial Intelligence.

## Latest Trends in Artificial Intelligence

The field of AI is constantly evolving, with new trends emerging to enhance its capabilities. One significant trend is the advancements in machine learning algorithms, enabling AI systems to learn and adapt to complex data patterns more efficiently. Additionally, there is a growing emphasis on explainable AI, where AI systems provide transparent explanations for their decisions, increasing trust and understanding among users. Ethical AI practices are also gaining traction, as companies strive to develop AI solutions that align with moral and social values. The integration of AI in key sectors such as healthcare, finance, and retail is another notable trend, showcasing the diverse applications of AI technology in optimizing processes and driving growth.

## Key Players in Artificial Intelligence

Several prominent companies are at the forefront of AI innovation, shaping the industry with their cutting-edge technologies. Google DeepMind stands out for its breakthroughs in deep learning and reinforcement learning, pushing the boundaries of AI research. IBM Watson is renowned for its cognitive computing capabilities, empowering businesses with AI-driven insights and solutions. Microsoft Azure AI offers a comprehensive suite of AI tools and services, enabling organizations to harness the power of AI for various applications. OpenAI is a leader in AI research and development, focusing on creating safe and beneficial AI for all. NVIDIA's expertise in GPU technology has revolutionized AI computing, driving advancements in AI research and implementation.

## Noteworthy News in Artificial Intelligence

Recent developments in natural language processing have showcased AI's ability to understand and generate human language with remarkable accuracy. The impact of AI on job automation and workforce transformation continues to be a topic of discussion, with AI technologies reshaping the labor market and driving new skill requirements. Ethical concerns surrounding AI technology, such as bias in algorithms and data privacy issues, highlight the importance of responsible AI development and deployment.

In conclusion, Artificial Intelligence is a dynamic and transformative field, with the latest trends and key players shaping its future trajectory. By staying informed on AI advancements and understanding the ethical implications of AI technology, businesses and individuals can navigate the evolving AI landscape with confidence and foresight. Whether you are a business professional seeking AI solutions, a tech enthusiast exploring AI innovations, or a student studying artificial intelligence, keeping abreast of the latest trends and key players in AI is essential in harnessing the full potential of this groundbreaking technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Exploring the Latest Trends and Key Players in the World of Digital Marketing

Outline:
I. Introduction
- Brief overview of the importance of digital marketing in today's business landscape
- Thesis statement highlighting the latest trends and key players in the industry

II. Latest Trends in Digital Marketing
A. Artificial Intelligence and Machine Learning
- Impact on personalized marketing strategies
- Examples of AI tools used in digital marketing

B. Video Marketing
- Rise of video content

I now can give a great answer

Final Answer:
# Exploring the Latest Trends and Key Players in the World of Digital Marketing

In today's rapidly evolving business landscape, digital marketing plays a crucial role in reaching and engaging with customers. As technology continues to advance, staying updated on the latest trends and key players in the digital marketing industry is essential for businesses looking to stay competitive. In this blog post, we will delve into the current trends shaping digital marketing strategies and highlight the key players driving innovation in the field.

## Latest Trends in Digital Marketing

### Artificial Intelligence and Machine Learning
Artificial Intelligence (AI) and Machine Learning have revolutionized the way businesses approach marketing. These technologies enable personalized marketing strategies by analyzing vast amounts of data to predict consumer behavior. AI tools such as chatbots, recommendation engines, and predictive analytics are increas

In [14]:
Markdown(result)

# Exploring the Latest Trends and Key Players in the World of Digital Marketing

In today's rapidly evolving business landscape, digital marketing plays a crucial role in reaching and engaging with customers. As technology continues to advance, staying updated on the latest trends and key players in the digital marketing industry is essential for businesses looking to stay competitive. In this blog post, we will delve into the current trends shaping digital marketing strategies and highlight the key players driving innovation in the field.

## Latest Trends in Digital Marketing

### Artificial Intelligence and Machine Learning
Artificial Intelligence (AI) and Machine Learning have revolutionized the way businesses approach marketing. These technologies enable personalized marketing strategies by analyzing vast amounts of data to predict consumer behavior. AI tools such as chatbots, recommendation engines, and predictive analytics are increasingly used in digital marketing to enhance customer experiences and drive conversions.

### Video Marketing
The rise of video content on social media platforms has transformed how brands connect with their audiences. Video marketing allows businesses to convey their messages in a visually engaging way, leading to higher engagement rates and increased brand awareness. Incorporating video into marketing campaigns has become a must-have strategy for businesses looking to stand out in a crowded digital landscape.

### Voice Search Optimization
With the growing popularity of voice-activated devices like smart speakers and virtual assistants, optimizing for voice search has become essential for businesses. Voice search optimization involves tailoring content to match natural language queries and providing concise, accurate answers. Implementing strategies to improve voice search visibility can help businesses capture a larger share of voice search traffic and enhance their online presence.

## Key Players in Digital Marketing

### Google
As the dominant player in the search engine market, Google wields significant influence over digital marketing strategies. Updates to Google algorithms can have a profound impact on website rankings and SEO performance. Staying informed about these changes and adapting marketing tactics accordingly is crucial for businesses looking to maintain visibility in search results.

### Facebook
Facebook's advertising platform remains a powerhouse in digital marketing, offering businesses a variety of targeting options and ad formats to reach their desired audience. Recent developments in Facebook's advertising platform, such as improved analytics and ad placement tools, provide marketers with the tools they need to create effective campaigns and drive conversions.

### Amazon
Amazon's advertising services have expanded rapidly in recent years, offering businesses unique opportunities to reach customers on the e-commerce giant's platform. Leveraging Amazon's advertising tools can help businesses increase product visibility, drive sales, and improve overall marketing performance. As Amazon continues to grow its advertising capabilities, businesses can capitalize on this platform to enhance their marketing strategies.

## Target Audience Analysis
This blog post is tailored to small to medium-sized businesses seeking to enhance their digital marketing strategies, marketing professionals looking to stay updated on industry trends, and entrepreneurs eager to capitalize on the latest marketing innovations. By providing insights into the latest trends and key players in digital marketing, we aim to empower our audience to make informed decisions and drive success in their marketing efforts.

## Call to Action
As you navigate the ever-changing landscape of digital marketing, it's essential to stay informed about the latest trends and innovations shaping the industry. Implement the insights shared in this blog post into your marketing strategies to stay ahead of the curve and drive meaningful results. For further reading and learning about digital marketing, check out the recommended resources below.

By embracing the latest trends and understanding the key players in digital marketing, businesses can position themselves for success in a competitive marketplace. Stay informed, stay innovative, and watch your digital marketing efforts thrive.

Resources:
- "The State of Digital Marketing in 2021" - HubSpot
- "AI in Digital Marketing: How AI is Reshaping the Industry" - Forbes
- "Video Marketing Statistics for 2021" - Wyzowl
- "Voice Search Optimization: A Guide for Marketers" - Search Engine Journal
- "The Rise of Amazon Advertising" - Digiday

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).